In [ ]:
pip install yfinance

In [ ]:
pip install fredapi

In [ ]:
import pandas as pd
import math
import numpy as np
import fredapi as fd
import yfinance as yf
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
fred = fd.Fred(api_key = 'bde7928ce1d3cc555b5d2fb725f0ef4b')

In [ ]:
cli = pd.DataFrame(fred.get_series('USALOLITONOSTSAM', frequency= 'q', aggregation_method= 'eop' ).dropna()) #GDP
ppi = pd.DataFrame(fred.get_series('PPIACO', units= 'pc1', frequency= 'q', aggregation_method= 'eop').dropna()) #Inflation
yc = pd.DataFrame(fred.get_series('T10Y2Y', units= 'lin', frequency= 'q', aggregation_method= 'avg').shift().dropna()) #Monetary Policy (Yeild Curve)
economic_data = pd.merge(cli, ppi, right_index = True, left_index = True)
economic_data = pd.merge(economic_data, yc, right_index = True, left_index = True)
economic_data.columns = ['CLI', 'PPI', 'YC']
economic_data.index.name = 'Date'
economic_data

In [ ]:
ticker = 'SPY'
start_date = '1980-05-22'
end_date = '2023-05-29'
interval = '3mo'
financial_data = yf.download(ticker, start = start_date, end = end_date, interval= interval)
financial_data = financial_data['Adj Close']
financial_data

In [ ]:
df = pd.merge(economic_data, financial_data, left_index = True, right_index =True )
df['Quarterly Return'] = df['Adj Close'].pct_change().mul(100)
df['CLIfd'] = df['CLI'].diff()
df['PPIfd'] = df['PPI'].diff()
df['ycfd'] = df['YC'].diff()
df = df.dropna()
df

In [ ]:
df['mean CLIfd'] = df['CLIfd'].rolling(8).mean()
df['std CLIfd'] = df['CLIfd'].rolling(8).std()
df['mean PPIfd'] = df['PPIfd'].rolling(8).mean()
df['std PPIfd'] = df['PPIfd'].rolling(8).std()
df = df.dropna()

In [ ]:
df['CLI z'] = ( df['CLIfd'] - df['mean CLIfd'] ) / df['std CLIfd']
df['PPI z'] = ( df['PPIfd'] - df['mean PPIfd'] ) / df['std PPIfd']

In [ ]:
df

In [ ]:
df['Decile CLI'] = pd.qcut(df['CLI z'], 5, labels=False)
df['Decile PPI'] = pd.qcut(df['PPI z'], 5, labels=False)

In [ ]:
df

In [ ]:
conditions= [
    (df["CLIfd"]>0) & (df["PPIfd"]>0),
    (df["CLIfd"]<0) & (df["PPIfd"]<0),
    (df["CLIfd"]>0) & (df["PPIfd"]<0),
    (df["CLIfd"]<0) & (df["PPIfd"]>0),
    ]

values = ['2','4','1','3']

df['Quad'] = np.select(conditions, values)
df

In [ ]:
conditions= [
    (df["ycfd"]>=0) & (df["YC"]<0),
    (df["ycfd"]<0) & (df["YC"]>0),
    (df["ycfd"]>=0) & (df["YC"]>0),
    (df["ycfd"]<0) & (df["YC"]<0),
     ]

values = ['fu','fd', 'su', 'in']

df['ycmc'] = np.select(conditions, values)
df

In [ ]:
df['QDYC'] = df['Quad'].astype('str')+","+ df['ycmc'].astype('str')
df

In [ ]:
df['QGrowth'] = df['Quad'].astype('str')+","+ df['Decile CLI'].astype('str')
df['QInflation'] = df['Quad'].astype('str')+","+ df['Decile PPI'].astype('str')

In [ ]:
df

In [ ]:
df.groupby(['QGrowth'])['Quarterly Return'].describe()

In [ ]:
df.groupby(['QInflation'])['Quarterly Return'].describe()

In [ ]:
df.groupby(['ycmc'])['Quarterly Return'].describe()

In [ ]:
df.groupby(['QDYC'])['Quarterly Return'].describe()

In [ ]:
df.groupby(['Quad'])['Quarterly Return'].describe()

In [ ]:
df.groupby(['Quad'])['Quarterly Return'].median()

In [ ]:
df.groupby(['Quad'])['Quarterly Return'].mean().mul(4)

In [ ]:
df.groupby(['Quad'])['Quarterly Return'].std().mul(math.sqrt(4))

In [ ]:
df['Positive Returns'] = (df['Quarterly Return']>0)
(df.groupby(['Quad'])['Positive Returns'].value_counts()/df.groupby(['Quad'])['Quarterly Return'].count()).mul(100)

In [ ]:
(df.groupby(['QDYC'])['Positive Returns'].value_counts()/df.groupby(['QDYC'])['Quarterly Return'].count()).mul(100)

In [ ]:
def addlabels(x,y):
  for i in range(len(x)):
    plt.text(i, y[i]+0.04, y[i], ha = 'center')

plt.figure(figsize=(6,6))
sharpe= pd.DataFrame(df.groupby(['Quad'])['Quarterly Return'].mean().mul(4)/df.groupby(['Quad'])['Quarterly Return'].std().mul(math.sqrt(4)))
plt.bar(sharpe.index, sharpe['Quarterly Return'])
addlabels(sharpe.index, sharpe['Quarterly Return'].round(2))

In [ ]:
df['Quarterly Return'].hist(by=df['Quad'])
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
df.boxplot(column=['Quarterly Return'], by= 'Quad', ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.violinplot(data=df, x="Quarterly Return", y="Quad", linewidth=2, ax=ax)
plt.show()

In [ ]:
df.groupby(['Quad'])['Quarterly Return'].skew()

Bayesian Backtest

In [ ]:
df['Shift'] = df['Quad'].shift().dropna()
df['Prior Quad'] = df['Shift'].astype('str')+","+ df['Quad'].astype('str')
df= df.dropna()
df

In [ ]:
df.groupby(['Prior Quad'])['Quarterly Return'].describe()

In [ ]:
def addlabels(x,y):
  for i in range(len(x)):
    plt.text(i, y[i]+0.04, y[i], ha = 'center')

plt.figure(figsize=(6,6))
psharpe= pd.DataFrame(df.groupby(['Prior Quad'])['Quarterly Return'].mean().mul(4)/df.groupby(['Prior Quad'])['Quarterly Return'].std().mul(math.sqrt(4)))
plt.bar(psharpe.index, psharpe['Quarterly Return'])
addlabels(psharpe.index, psharpe['Quarterly Return'].round(2))

In [ ]:
(df.groupby(['Prior Quad'])['Positive Returns'].value_counts()/df.groupby(['Prior Quad'])['Quarterly Return'].count()).mul(100)